In [108]:
import pandas as pd
import openai
from dotenv import load_dotenv
load_dotenv()
import os
from langchain.prompts import load_prompt
from langchain import PromptTemplate
from langchain.llms import OpenAIChat
import sys
pd.options.mode.chained_assignment = None
from langchain.callbacks import get_openai_callback
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import TextLoader
from langchain import PromptTemplate, LLMChain, VectorDBQA
from langchain.chains import TransformChain, SequentialChain
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.agents import initialize_agent, Tool
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.agents import initialize_agent
import re
import json
import stripe

from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    SystemMessage
)
import asyncio


import sys
sys.path.append('./modules/')
openai.api_key = os.getenv('OPENAI_KEY')
key = os.getenv('OPENAI_KEY')
serpi_api_key = os.getenv('SERPAPI_API_KEY')

from Generator import Generator
from Extractor import Extractor
from PromptPath import PromptPath
from GenExtract import GenExtract
from Property import Property
from NaturalLanguagePool import NaturalLanguagePool
from DynamicChatHistory import DynamicChatHistory
from Conversation import Conversation
from ConversationScript import ConversationScript
from ConversationScriptStage import ConversationScriptStage
from show_chat_log import show_chat_log
from is_question import is_question
from customer_profile import customer_profile

times_embedding_run = 10

RED = "\033[91m"
GREEN = "\033[92m"
RESET = "\033[0m"
BOLD = "\033[1m"

llm = ChatOpenAI(openai_api_key=key)

In [2]:
from langchain import OpenAI, SerpAPIWrapper


In [3]:
#load in the policies to the vectorstore
loaders = []
docs = []

#create a list with the names of the files to be loaded in the data/policies folder
for file in os.listdir('data/policies'):
    if file.endswith('.txt'):
        loader = TextLoader('data/policies/' + file)
        loaders.append(loader)

for loader in loaders:
    docs.extend(loader.load())

text_splitter = CharacterTextSplitter(chunk_size=250, chunk_overlap=0)
documents = text_splitter.split_documents(docs)

embeddings = OpenAIEmbeddings(openai_api_key=key)
policies = Chroma.from_documents(documents, embeddings)
times_embedding_run += 1

Created a chunk of size 408, which is longer than the specified 250
Created a chunk of size 377, which is longer than the specified 250
Created a chunk of size 293, which is longer than the specified 250
Created a chunk of size 298, which is longer than the specified 250
Created a chunk of size 332, which is longer than the specified 250
Created a chunk of size 510, which is longer than the specified 250
Created a chunk of size 514, which is longer than the specified 250
Created a chunk of size 350, which is longer than the specified 250
Created a chunk of size 265, which is longer than the specified 250
Created a chunk of size 458, which is longer than the specified 250
Created a chunk of size 667, which is longer than the specified 250
Created a chunk of size 282, which is longer than the specified 250
Created a chunk of size 364, which is longer than the specified 250
Created a chunk of size 441, which is longer than the specified 250
Created a chunk of size 360, which is longer tha

Running Chroma using direct local API.
Using DuckDB in-memory for database. Data will be transient.


In [109]:
from langchain.agents import tool

@tool
def get_business_knowledge(query):
    "Retrieves information about the businsess from the knowledge base based on a question about the business."
    policies_results = policies.similarity_search(query)
    policy_text = '\n'.join([nugget.page_content for nugget in policies_results])
    print(policy_text)

    template = """Answer the question to your best ability based on the following policies. If you don't know, just say I don't know. Create a clear and succinct explaination of the policies.
    Question: {query}
    Policies: {policy_text}"""
    
    prompt = PromptTemplate(template=template, input_variables=['query', 'policy_text'])
    
    chain = LLMChain(
        prompt=prompt,
        llm=llm
    )

    response = chain.run({'query': query, 'policy_text': policy_text})
    return response
    

In [110]:
tools = [
    Tool(
        name="Get Business Knowledge",
        func=get_business_knowledge.run,
        description="use this any time someone asks a question about what your business offers or you have any uncertainty about what your business offers or it's policies"
    )
]

In [111]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent="zero-shot-react-description",
    verbose=True
)

In [112]:
template = """You are a helpful sales agent communicating with a prospect for Better Termite & Pest Control through a chat window. You should come off as cool, relaxed and informal, but helpful. A new lead has reached out through our site. I will provide you with goals based on the information you have collected so far. Try to avoid asking multiple questions in one response. Avoid overly formal words and phrases."""

memory = ConversationBufferMemory(memory_key="chat_history")
memory.chat_memory.messages.append(SystemMessage(content=template))

agent_chain = initialize_agent(tools, llm, agent="conversational-react-description", verbose=True, memory=memory)

In [113]:
agent_chain.run(input="hi, i am bob")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Hi Bob, nice to meet you! How can I assist you with your pest control needs today?

> Finished chain.


'Hi Bob, nice to meet you! How can I assist you with your pest control needs today?'

In [114]:
agent_chain.run('Do you offer pest control services?')



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get Business Knowledge
Action Input: Pest Control ServicesAt our company, we strive to provide prompt and efficient pest control services to all of our customers. To ensure that our scheduling process runs smoothly, we have implemented the following policies:
Quarterly Pest Control Program: Offers pest control services, 4 visits per year.
Our treatments are designed to be effective and safe for humans and pets.

We offer ongoing pest control services, including routine treatments to prevent future infestations.
Quarterly Pest Control Guarantee: We believe customers expect prompt, friendly and effective service. If you are ever unhappy with your service, your plan covers unlimited free callbacks in between services. If we can't find a way to make you happy, you can cancel at anytime.

Observation: Answer: The company provides pest control services to their customers and has implemented policies to ensu

ValueError: Could not parse LLM output: `Do I need to use a tool? No`

In [115]:
agent_chain.run(input="Do you offer wildlife control services?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? Yes
Action: Get Business Knowledge
Action Input: "Do you offer wildlife control services?"While we strive to provide comprehensive pest control services, we do not cover bed bugs, wildlife, or other animal infestations. Customers with these issues should contact qualified professionals who specialize in bed bug treatments or wildlife removal. Our technicians are available to provide recommendations and assistance with identifying these issues. If you have any questions or concerns regarding bed bugs, wildlife, or other animals, please contact us at 703-683-2000.
Our technicians are trained to identify bed bugs and other wildlife infestations and will provide recommendations for customers to address the issue.

We do not provide any treatments or services for bed bugs, wildlife, or other animals.
Our pest control services do not cover bed bugs, any type of wildlife (including squirrels, raccoons, and snakes), or a

ValueError: Could not parse LLM output: `Do I need to use a tool? No`

In [107]:
agent_chain.run(input="Would you be able to help me with squirrels?")



> Entering new AgentExecutor chain...
Thought: Do I need to use a tool? No
AI: Absolutely, we can help with squirrel problems too! Our pest control technicians are trained to safely and effectively remove squirrels from your property and prevent future squirrel infestations. Would you like to schedule a service or do you have any other questions?

> Finished chain.


'Absolutely, we can help with squirrel problems too! Our pest control technicians are trained to safely and effectively remove squirrels from your property and prevent future squirrel infestations. Would you like to schedule a service or do you have any other questions?'

In [2]:
from dotenv import load_dotenv
load_dotenv()
import os

import sys
sys.path.append('./modules/')
key = os.getenv('OPENAI_KEY')
from ChooseService import analyze_situation_sequential


Running Chroma using direct local API.
loaded in 52 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection
Running Chroma using direct local API.
loaded in 11 embeddings
loaded in 1 collections
collection with name langchain already exists, returning existing collection


In [3]:
conversation = """
Lead: I am looking for a one time ant treatment.
Agent: Oh no! I'm sorry to hear that John. Luckily we can help with that :) Would you mind if I asked you a few questions so I can get a handle on your issue?
Lead: Sure, go ahead.
Agent: Great thanks! First how long has this been going on?
Lead: 2 Weeks
Agent: And how many ants are you seeing?
Lead: A lot! They are all over the kitchen.
Agent: OK don't worry I see this kind of thing all the time. How big is your house?
Lead: 2000 square feet
Agent: Got it. Give me just a second and let me run some numbers for you.
Agent: OK here's what I'm thinking:
"""

analyze_situation_sequential({
    "property_type": "single family",
    "square_footage": 2000,
    "acres": 1,
    "target": "ants",
    "severity": "high",
    "conversation_history": conversation
})



> Entering new SequentialChain chain...
STEP: Getting situational experiences
Situational experiences found:  ['Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.', 'Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.', 'Ants will frequently come back and customers will benefit from ongoing services. One time services are not advisable but can be offered only if the customer doesn’t want a plan.', 'Ants will frequently come back and customers will benefit from ongoing services. One time services are not advisable but can be offered only if the customer doesn’t want a plan.', 'Millipedes can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow cust

{'property_type': 'single family',
 'square_footage': 2000,
 'acres': 1,
 'target': 'ants',
 'severity': 'high',
 'conversation_history': "\nLead: I am looking for a one time ant treatment.\nAgent: Oh no! I'm sorry to hear that John. Luckily we can help with that :) Would you mind if I asked you a few questions so I can get a handle on your issue?\nLead: Sure, go ahead.\nAgent: Great thanks! First how long has this been going on?\nLead: 2 Weeks\nAgent: And how many ants are you seeing?\nLead: A lot! They are all over the kitchen.\nAgent: OK don't worry I see this kind of thing all the time. How big is your house?\nLead: 2000 square feet\nAgent: Got it. Give me just a second and let me run some numbers for you.\nAgent: OK here's what I'm thinking:\n",
 'situational_experience': ['Ants can be well managed by bimonthly service frequencies. It is less advisable, but still an option to allow customers to use quarterly services. Monthly services are often overkill for ant issues.',
  'Ants c